In [121]:
from wilcoxon.spiderman import *
import re
from tqdm import tqdm
import pandas as pd
from wilcoxon import sheets
from wilcoxon.language import *

In [10]:
subsite = website(links[0])

In [92]:
def JLPTKanji():
    links = []
    i = 1
    while i < 12:
        try:
            site = website("https://jlptsensei.com/kanji-list-ordered-by-frequency-of-use/page/{}/".format(i))
            links.extend(list(set(a["href"] for a in site.html("a", href=re.compile("learn-japanese-kanji")))))
            i += 1
        except:
            break
    links = list(set(links))

    result = []

    for link in tqdm(links):

        subsite = website(link)

        try: kpro = allstr(subsite.html(class_="header-kunyomi")[0]," ")
        except: kpro = ""
        try: opro = allstr(subsite.html(class_="header-onyomi")[0]," ")
        except: opro = ""

        krea = []
        try:
            for word in list(subsite.html(text=re.compile("Kunyomi Readings"))[0].next_siblings):
                try:
                    krea.append(tuple(map(lambda x: x.strip(), re.split("【|】",word))))
                except: continue
        except: None            

        orea = []
        try:
            for word in list(subsite.html(text=re.compile("Onyomi Readings"))[0].next_siblings):
                try:
                    orea.append(tuple(map(lambda x: x.strip(), re.split("【|】",word))))
                except: continue
        except: None

        sentences = []
        for h in subsite.html("h5"):
            try:
                jap, *a, romaji, meaning = list(map(allstr, h.next_siblings))
                sentences.append((jap, romaji, meaning))
            except: continue

        try: rank = int(allstr(subsite.html(text="2,500")[0].find_previous("b").find_previous("b")))
        except: rank = 0
        try: meaning = subsite.html(text=re.compile("Meaning: "))[0]
        except: meaning = ""
        try: kanji = subsite.html(text=re.compile("Meaning of"))[0].replace("Meaning of","").strip()
        except: kanji = ""

        for sentence in sentences:
            try:
                result.append({"Rank":rank,
                               "Kanji":kanji,
                               "Word":kanji,
                               "KPro":kpro,
                               "OPro":opro,
                               "Meaning":meaning,
                               "Sentence":sentence[0],
                               "Romaji":sentence[1],
                               "SMeaning":sentence[2]})
            except: None

        for kre in krea:
            try:
                result.append({"Rank":rank,
                               "Kanji":kanji,
                               "Word":kre[0],
                               "KPro":kre[1],
                               "OPro":"",
                               "Meaning":kre[2],
                               "Sentence":"",
                               "Romaji":"",
                               "SMeaning":""})
            except: None

        for ore in orea:
            try:
                result.append({"Rank":rank,
                               "Kanji":kanji,
                               "Word":ore[0],
                               "KPro":ore[1],
                               "OPro":"",
                               "Meaning":ore[2],
                               "Sentence":"",
                               "Romaji":"",
                               "SMeaning":""})
            except: None
    return result

100%|██████████| 1032/1032 [10:32<00:00,  1.63it/s]


In [95]:
result

[{'Rank': 268,
  'Kanji': '電',
  'Word': '電',
  'KPro': '',
  'OPro': 'デン den',
  'Meaning': 'Meaning: electricity; electric powered.',
  'Sentence': 'その電気を消してもいいですか？',
  'Romaji': 'sono denki wo keshitemo ii desu ka?',
  'SMeaning': 'Is it OK if I turn off that light?'},
 {'Rank': 268,
  'Kanji': '電',
  'Word': '電',
  'KPro': '',
  'OPro': 'デン den',
  'Meaning': 'Meaning: electricity; electric powered.',
  'Sentence': '田中さんに電話してください。',
  'Romaji': 'tanaka san ni denwa shite kudasai.',
  'SMeaning': 'Please call Tanaka-san.'},
 {'Rank': 268,
  'Kanji': '電',
  'Word': '電',
  'KPro': '',
  'OPro': 'デン den',
  'Meaning': 'Meaning: electricity; electric powered.',
  'Sentence': '学校に電車で行きます。',
  'Romaji': 'gakkou ni densha de ikimasu.',
  'SMeaning': 'I go to school by train.'},
 {'Rank': 268,
  'Kanji': '電',
  'Word': '電',
  'KPro': '',
  'OPro': 'デン den',
  'Meaning': 'Meaning: electricity; electric powered.',
  'Sentence': 'あなたの家も停電していますか？',
  'Romaji': 'anata no ie mo teiden shitei masu

In [103]:
kanji = sheets.getVocab("Kanji")

In [151]:
kanji.OPro.apply(tryRomaji).fillna("").apply(
    lambda x: x.replace(" ( ","(")
).apply(
    lambda x: ", ".join(list(set([a for a in re.split("[、, ]",x) if len(a) > 0])))
).to_clipboard(index=False)